In [48]:
# -*- coding: utf-8 -*-
"""
generate_invoices.py  – FINAL (payment_type + phone/PAN + declaration under PAN)
• Address lines 9 pt
• Extra padding (name/address, meta block)
• Thin grey dividers
• Bold guaranteed via Arial-Bold or DejaVuSans-Bold
• NOTE block is always TWO lines; big amount stays flush-right
• payment_type = "marketplace" | "submerchant"
• BILL TO block shows student Phone, PAN, and Declaration (if provided)
"""

import os, unicodedata, datetime as _dt
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.lib import colors
from reportlab.platypus import Table, TableStyle, Paragraph
from reportlab.lib.styles import ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont


# ────────── Font setup ──────────
def reg(alias, path):
    if os.path.exists(path):
        pdfmetrics.registerFont(TTFont(alias, path))
        return True
    return False


reg("Arial", "arial.ttf")
reg("Arial-Bold", "arialbd.ttf")
reg("DejaVuSans", "DejaVuSans.ttf")
reg("DejaVuSans-Bold", "DejaVuSans-Bold.ttf")

BASE_FONT = "Arial" if "Arial" in pdfmetrics.getRegisteredFontNames() else "DejaVuSans"
BOLD_FONT = (
    "Arial-Bold"
    if "Arial-Bold" in pdfmetrics.getRegisteredFontNames()
    else "DejaVuSans-Bold"
)
RUPEE_FONT = "DejaVuSans"  # contains ₹

# ────────── Config ──────────
# Choose settlement model — "marketplace" (two invoices) or "submerchant" (single combined teacher invoice)
payment_type = "submerchant"  # <-- change to "submerchant" for single combined invoice

# ────────── Data ──────────
transaction_ID = "4168056958"

student_name = "VENNA VENUGOPALA REDDY"
student_phone = "9441803301"                 # optional: "" or None to hide
student_pan   = "BSNPV9888F"                 # optional: "" or None to hide
education_fees_declaration = "Education fees Declaration : YES"   # optional: "" or None to hide

teacher_name = "VENNA PALANKA VISHNU VARDHAN REDDY"
teacher_addr1 = "No. 8-2-120/112, 1st Floor, Park View Estate, Road No. 2, Banjara Hills"
teacher_city = "Hyderabad"
teacher_state = "Telangana"
teacher_pin = "500034"

company_name = "Mateira Technologies Private Limited"
company_addr1 = "29 R R Road"
company_addr2 = "Ghaziabad, Uttar Pradesh"
company_addr3 = "India – 201001"

service_amount = 10000.00
platform_fees = 119.41
transaction_date = [2025, 7, 25]  # YYYY, M, D
invoice_date = _dt.datetime(*transaction_date).strftime("%d %b %Y")
invoice_teacher_no = f"#{transaction_date[0]}{transaction_date[1]:02d}{transaction_date[2]:02d}/026" # sequential for the txns on that day
invoice_company_no = f"#{transaction_date[0]}{transaction_date[1]:02d}{transaction_date[2]:02d}/025"

RUPEE = "\u20b9"

# ────────── Layout constants ──────────
PAGE_W, PAGE_H = A4
MARGIN = 10 * mm
CONTENT_W = PAGE_W - 2 * MARGIN
RULE_MARGIN = 5 * mm
TITLE_Y = PAGE_H - 30 * mm
NAME_ADDR_GAP = 7 * mm
META_LINE_HT = 5 * mm
AMT_GAP_Y = 10 * mm

p9 = ParagraphStyle("p9", fontName=BASE_FONT, fontSize=9, leading=11)


# ────────── Helper functions ──────────
def clean(txt: str):
    unwanted = (0x00A0, 0x202F, 0x2007, 0x2060)
    return unicodedata.normalize("NFKC", txt).translate(dict.fromkeys(unwanted, 0x20))


def money(v):
    return f"{RUPEE}{v:,.2f}"


def right(c, txt, xr, y, font, size, color=colors.black):
    txt = clean(txt)
    c.setFillColor(color)
    c.setFont(font, size)
    c.drawRightString(xr, y, txt)
    c.setFillColor(colors.black)


def hline(c, y):
    c.setStrokeColor(colors.grey)
    c.setLineWidth(0.3)
    c.line(RULE_MARGIN, y, PAGE_W - RULE_MARGIN, y)
    c.setStrokeColor(colors.black)
    c.setLineWidth(1)


def make_tbl(data, widths, style):
    tbl = Table(data, colWidths=widths, hAlign="LEFT")
    tbl.setStyle(TableStyle(style))
    return tbl


def draw_tbl(c, tbl, x, y):
    _, h = tbl.wrapOn(c, CONTENT_W, 0)
    tbl.drawOn(c, x, y - h)
    return h


# ────────── Section builders ──────────
def bill_meta(c, y_top, inv_no):
    """
    Left block: BILL TO + student name + phone + PAN + (declaration under PAN)
    Right block: invoice metadata (invoice #, dates)
    """
    bill_rows = [["BILL TO", ":"], [student_name, ""]]

    # Optional rows under the name
    extra_rows = []
    if student_phone:
        extra_rows.append([f"+91-{clean(student_phone)}", ""])
    if student_pan:
        extra_rows.append([f"PAN: {clean(student_pan)}", ""])
    if education_fees_declaration:
        extra_rows.append([clean(education_fees_declaration), ""])

    bill_rows.extend(extra_rows)

    tbl = make_tbl(
        bill_rows,
        [25 * mm, CONTENT_W - 25 * mm],
        [
            # Row 0: "BILL TO"
            ("FONTNAME", (0, 0), (0, 0), BOLD_FONT),
            ("FONTSIZE", (0, 0), (0, 0), 8),

            # Row 1: Student name (bold, bigger)
            ("FONTNAME", (0, 1), (0, 1), BOLD_FONT),
            ("FONTSIZE", (0, 1), (0, 1), 11),

            # Remaining rows: phone, PAN, declaration (normal)
            ("FONTNAME", (0, 2), (-1, -1), BASE_FONT),
            ("FONTSIZE", (0, 2), (-1, -1), 9),
            ("TOPPADDING",  (0, 2), (-1, -1), 1.5),
            ("BOTTOMPADDING", (0, 2), (-1, -1), 1.5),
            ("LEFTPADDING", (0, 0), (-1, -1), 2),
            ("RIGHTPADDING", (0, 0), (-1, -1), 2),
        ],
    )

    h_tbl = tbl.wrapOn(c, CONTENT_W, 0)[1]
    meta_x = PAGE_W - MARGIN - 2 * mm
    block_h = max(h_tbl, 6 * META_LINE_HT)

    # draw left table
    tbl.drawOn(c, MARGIN, y_top - h_tbl)

    # draw right metadata (invoice #, dates)
    y = y_top
    for head, val in (
        ("INVOICE #", inv_no),
        ("DATE", invoice_date),
        ("INVOICE DUE DATE", invoice_date),
    ):
        right(c, head, meta_x, y, BOLD_FONT, 8)
        y -= META_LINE_HT
        right(c, val, meta_x, y, BASE_FONT, 8)
        y -= META_LINE_HT

    return y_top - block_h - 5 * mm


def items_tbl(c, y, row):
    widths = [40 * mm, 64 * mm, 20 * mm, 33 * mm, 33 * mm]
    data = [["SERVICES", "DESCRIPTION", "QTY", "UNIT PRICE", "AMOUNT"], row]
    style = [
        ("FONTNAME", (0, 0), (-1, 0), BOLD_FONT),
        ("FONTSIZE", (0, 0), (-1, 0), 8),
        ("FONTNAME", (0, 1), (-1, 1), BASE_FONT),
        ("FONTSIZE", (0, 1), (-1, 1), 9),
        ("ALIGN", (2, 0), (-1, -1), "RIGHT"),
        ("LEFTPADDING", (0, 0), (-1, -1), 2),
        ("RIGHTPADDING", (0, 0), (-1, -1), 2),
    ]
    y -= draw_tbl(c, make_tbl(data, widths, style), MARGIN, y) + 10
    hline(c, y)
    return y - 8 * mm


def note_total(c, y, amt):
    rx = PAGE_W - MARGIN - 2 * mm
    # Row 1 labels
    c.setFont(BOLD_FONT, 8)
    c.drawString(MARGIN, y, "NOTE:")
    right(c, "TOTAL", rx, y, BOLD_FONT, 8)
    # Row 2 (part 1) note wrap + amount
    y -= AMT_GAP_Y
    c.setFont(BASE_FONT, 9)
    c.drawString(
        MARGIN, y, "Credit Card standard settlement transactions take up to 24 hours"
    )
    right(c, money(amt), rx, y, RUPEE_FONT, 26)
    # Row 3 (part 2) remainder of note
    y -= 4 * mm
    c.setFont(BASE_FONT, 9)
    c.drawString(MARGIN, y, "for amount settlements.")


# ────────── Invoice A: Student → Teacher ──────────
def build_teacher(
    fn,
    *,
    inv_no=None,
    line_title=None,
    line_desc=None,
    unit_amt=None,
    qty="1",
    total_amt=None,
):
    """
    Flexible teacher invoice builder.
    Defaults to the original single-line 'Educational Service' row when args are None.
    """
    inv_no = inv_no or invoice_teacher_no
    line_title = line_title or "Educational Service"
    line_desc = line_desc or "Service rendered by the teacher to the student"
    unit_amt = unit_amt if unit_amt is not None else service_amount
    total_amt = total_amt if total_amt is not None else service_amount

    c = Canvas(fn, A4)
    c.setFont(BOLD_FONT, 36)
    c.drawString(MARGIN, TITLE_Y, "Invoice")
    hdr_x = PAGE_W - MARGIN - 2 * mm
    right(
        c,
        "Generated by Mateira Technologies Private Limited on the behalf of",
        hdr_x,
        TITLE_Y,
        BASE_FONT,
        8,
        colors.grey,
    )
    y = TITLE_Y - 8 * mm
    right(c, teacher_name, hdr_x, y, BOLD_FONT, 11)
    y -= NAME_ADDR_GAP
    for line in (teacher_addr1, f"{teacher_city}, {teacher_state} - {teacher_pin}"):
        right(c, line, hdr_x, y, BASE_FONT, 9)
        y -= 5 * mm
    hline(c, y - 3 * mm)
    y -= 9 * mm
    y = bill_meta(c, y, inv_no)
    hline(c, y)
    y -= 10 * mm

    row = [
        Paragraph(clean(line_title), p9),
        Paragraph(clean(line_desc), p9),
        str(qty),
        money(unit_amt),
        money(total_amt),
    ]
    y = items_tbl(c, y, row)
    note_total(c, y, total_amt)
    c.save()
    print(f"✓ {os.path.basename(fn)}")


# ────────── Invoice B: Student → Company ──────────
def build_company(fn):
    c = Canvas(fn, A4)
    c.setFont(BOLD_FONT, 36)
    c.drawString(MARGIN, TITLE_Y, "Invoice")
    hdr_x = PAGE_W - MARGIN - 2 * mm
    right(c, company_name, hdr_x, TITLE_Y, BOLD_FONT, 11)
    y = TITLE_Y - 8 * mm - 2 * mm  # extra gap after company name
    for line in (company_addr1, company_addr2, company_addr3):
        right(c, line, hdr_x, y, BASE_FONT, 9)
        y -= 5 * mm
    hline(c, y - 3 * mm)
    y -= 9 * mm
    y = bill_meta(c, y, invoice_company_no)
    hline(c, y)
    y -= 10 * mm
    row = [
        Paragraph("Eira.club platform fees", p9),
        Paragraph("Fees for processing payments through Eira.club app", p9),
        "1",
        money(platform_fees),
        money(platform_fees),
    ]
    y = items_tbl(c, y, row)
    note_total(c, y, platform_fees)
    c.save()
    print(f"✓ {os.path.basename(fn)}")


# ────────── Build PDFs based on payment_type ──────────
if payment_type.lower() == "marketplace":
    # Original behavior: 2 invoices
    build_teacher(
        f'{invoice_teacher_no.replace("/", "-")}_invoice_teacher-{transaction_ID if transaction_ID else ""}.pdf'
    )
    build_company(
        f'{invoice_company_no.replace("/", "-")}_invoice_company-{transaction_ID if transaction_ID else ""}.pdf'
    )

elif payment_type.lower() == "submerchant":
    # Single teacher→student invoice with combined amount (service + platform fees)
    combined_total = round((service_amount or 0) + (platform_fees or 0), 2)
    build_teacher(
        f'{invoice_teacher_no.replace("/", "-")}_invoice_teacher-{transaction_ID if transaction_ID else ""}.pdf',
        inv_no=invoice_teacher_no,
        line_title="Educational Service (incl. platform fees)",
        line_desc="Service rendered by the teacher to the student; includes Eira.club processing fees",
        unit_amt=combined_total,
        qty="1",
        total_amt=combined_total,
    )
else:
    raise ValueError("payment_type must be either 'marketplace' or 'submerchant'")


✓ #20250725-026_invoice_teacher-4168056958.pdf
